# 로지스틱 회귀 구현
---

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)

In [4]:
X_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
X_train = torch.FloatTensor(X_data)
y_train = torch.FloatTensor(y_data)

print(X_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [5]:
w = torch.zeros((2, 1), requires_grad= True)
b = torch.zeros(1, requires_grad= True)

In [7]:
h = 1 / (1 + torch.exp(-(X_train.matmul(w) + b)))
print(h)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


In [8]:
print(h)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [14]:
# 하나의 원소에 대한 오차
-(y_train[0] * torch.log(h[0])) + (1 - y_train[0]) * torch.log(1 - h[0])

tensor([-0.6931], grad_fn=<AddBackward0>)

In [13]:
# 모든 원소에 대한 오차
losses = -(y_train * torch.log(h) + (1-y_train) * torch.log(1 - h))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [15]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [16]:
# 로지스틱 회귀의 비용함수
F.binary_cross_entropy(h, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [18]:
X_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(X_data)
y_train = torch.FloatTensor(y_data)

In [20]:
w = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([w,b], lr=1)

epochs = 1000
for epoch in range(epochs + 1) :
    h = torch.sigmoid(X_train.matmul(w) + b)
    cost = -(y_train * torch.log(h) + (1- y_train) * torch.log(1- h)).mean() 
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))
    
    

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [21]:
h = torch.sigmoid(X_train.matmul(w) + b)
print(h)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [22]:
pred = h >= torch.FloatTensor([0.5])
print(pred)

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [23]:
print(w, b)

tensor([[3.2530],
        [1.5179]], requires_grad=True) tensor([-14.4819], requires_grad=True)


# nn.Module로 로지스틱 회귀 구현
---

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)

In [25]:
X_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
X_train = torch.FloatTensor(X_data)
y_train = torch.FloatTensor(y_data)

In [26]:
model = nn.Sequential( # module 층 쌓기
    nn.Linear(2, 1),
    nn.Sigmoid()
    )

In [27]:
model(X_train)

tensor([[0.5427],
        [0.6331],
        [0.4456],
        [0.6306],
        [0.6294],
        [0.5362]], grad_fn=<SigmoidBackward0>)

In [28]:
optimizer = optim.SGD(model.parameters(), lr=1)

epochs = 1000
for epoch in range(epochs + 1) :
    h = model(X_train)
    cost = F.binary_cross_entropy(h, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0 :
        pred = h >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True
        correct_pred = pred.float() == y_train # 실제값과 일치하면 True
        acc = correct_pred.sum().item() / len(correct_pred) # 정확도 계산
        print('Epoch {:4d}/{} Cost : {:.6f} Acc {:2.2f}%'.format(
        epoch, epochs, cost.item(), acc*100))

Epoch    0/1000 Cost : 0.653724 Acc 66.67%
Epoch   10/1000 Cost : 0.357625 Acc 83.33%
Epoch   20/1000 Cost : 0.428173 Acc 83.33%
Epoch   30/1000 Cost : 0.340048 Acc 83.33%
Epoch   40/1000 Cost : 0.286195 Acc 83.33%
Epoch   50/1000 Cost : 0.238611 Acc 100.00%
Epoch   60/1000 Cost : 0.196563 Acc 100.00%
Epoch   70/1000 Cost : 0.165590 Acc 100.00%
Epoch   80/1000 Cost : 0.148768 Acc 100.00%
Epoch   90/1000 Cost : 0.138111 Acc 100.00%
Epoch  100/1000 Cost : 0.129119 Acc 100.00%
Epoch  110/1000 Cost : 0.121249 Acc 100.00%
Epoch  120/1000 Cost : 0.114300 Acc 100.00%
Epoch  130/1000 Cost : 0.108120 Acc 100.00%
Epoch  140/1000 Cost : 0.102588 Acc 100.00%
Epoch  150/1000 Cost : 0.097606 Acc 100.00%
Epoch  160/1000 Cost : 0.093097 Acc 100.00%
Epoch  170/1000 Cost : 0.088996 Acc 100.00%
Epoch  180/1000 Cost : 0.085250 Acc 100.00%
Epoch  190/1000 Cost : 0.081814 Acc 100.00%
Epoch  200/1000 Cost : 0.078651 Acc 100.00%
Epoch  210/1000 Cost : 0.075730 Acc 100.00%
Epoch  220/1000 Cost : 0.073023 Acc 1

In [29]:
model(X_train)

tensor([[2.7247e-04],
        [3.1432e-02],
        [3.8745e-02],
        [9.5646e-01],
        [9.9825e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)

In [30]:
print(list(model.parameters()))

[Parameter containing:
tensor([[3.2587, 1.5210]], requires_grad=True), Parameter containing:
tensor([-14.5085], requires_grad=True)]


# 클래스로 구현
---

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)

In [4]:
class BinaryClassifier(nn.Module) :
    def __init__(self) :
        super().__init__() 
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x) :
        return self.sigmoid(self.linear(x))

In [5]:
model = BinaryClassifier()

In [8]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
X_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [9]:
optimizer = optim.SGD(model.parameters(), lr=1)

epochs = 1000
for epoch in range(epochs+1) :
    h = model(X_train)
    cost = F.binary_cross_entropy(h, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0 :
        p = h >= torch.FloatTensor([0.5])  # 예측값이 0.5 를 넘어야 True
        corr_pred = p.float() == y_train  # 실제값이 일치하는 경우만 True
        acc = corr_pred.sum().item() / len(corr_pred)
        print('Epoch {:4d}/{} Cost : {:.6f} Accuracy {:2.2f}%'.format(
        epoch, epochs, cost.item(), acc * 100))

Epoch    0/1000 Cost : 0.653724 Accuracy 66.67%
Epoch   10/1000 Cost : 0.357625 Accuracy 83.33%
Epoch   20/1000 Cost : 0.428173 Accuracy 83.33%
Epoch   30/1000 Cost : 0.340048 Accuracy 83.33%
Epoch   40/1000 Cost : 0.286195 Accuracy 83.33%
Epoch   50/1000 Cost : 0.238611 Accuracy 100.00%
Epoch   60/1000 Cost : 0.196563 Accuracy 100.00%
Epoch   70/1000 Cost : 0.165590 Accuracy 100.00%
Epoch   80/1000 Cost : 0.148768 Accuracy 100.00%
Epoch   90/1000 Cost : 0.138111 Accuracy 100.00%
Epoch  100/1000 Cost : 0.129119 Accuracy 100.00%
Epoch  110/1000 Cost : 0.121249 Accuracy 100.00%
Epoch  120/1000 Cost : 0.114300 Accuracy 100.00%
Epoch  130/1000 Cost : 0.108120 Accuracy 100.00%
Epoch  140/1000 Cost : 0.102588 Accuracy 100.00%
Epoch  150/1000 Cost : 0.097606 Accuracy 100.00%
Epoch  160/1000 Cost : 0.093097 Accuracy 100.00%
Epoch  170/1000 Cost : 0.088996 Accuracy 100.00%
Epoch  180/1000 Cost : 0.085250 Accuracy 100.00%
Epoch  190/1000 Cost : 0.081814 Accuracy 100.00%
Epoch  200/1000 Cost : 0.